## AI Assistant API Agents

Yandex Cloud contains native API for creating RAG-based intelligent agents that support both information retrieval and arbitrary function calling tools. The easiest way to use AI Assistant API is through a library called [Yandex Cloud ML SDK](https://yandex.cloud/ru/docs/foundation-models/sdk/). Its [GitHub Repo](https://github.com/yandex-cloud/yandex-cloud-ml-sdk) contains numerous examples of using it.

First, let's install it: 

In [ ]:
%pip install yandex-cloud-ml-sdk pydantic

We will assume that credentials required to run Yandex Foundation Models are stored in environment variables `folder_id` and `api_key` (eg. Datasphere Secrets or dotenv). Here is how we can start using YandexGPT model through SDK:

In [1]:
import os
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)

# Uncomment this, if you want to see what SDK is doing under the hood
# sdk.setup_default_logging(log_level='DEBUG')

model = sdk.models.completions("yandexgpt", model_version="rc")

In [2]:
from IPython.display import Markdown, display

def printx(string):
    display(Markdown(string))

printx(model.run("Which wine is best with steak?").text)

When pairing wine with steak, several options can complement the flavors and texture of the meat. Here are some popular choices:

1. **Cabernet Sauvignon**: Known for its bold, full-bodied flavor and firm tannins, Cabernet Sauvignon pairs well with the richness of steak. The wine's structure can stand up to the intensity of the meat.

2. **Merlot**: Often softer and more approachable than Cabernet Sauvignon, Merlot offers a balance of fruitiness and structure that pairs nicely with steak. It can be a good choice for those who prefer a slightly less tannic wine.

3. **Bordeaux**: A blend of grapes including Cabernet Sauvignon and Merlot, Bordeaux wines are known for their complexity and depth. They can offer a sophisticated pairing with steak, especially with well-aged steaks.

4. **Malbec**: Originating from Argentina, Malbec is known for its dark fruit flavors and smooth texture. It pairs well with steak, offering a rich and satisfying experience.

5. **Zinfandel**: With its bold fruit flavors and spiciness, Zinfandel can be an excellent match for steak, especially for those who enjoy a more fruit-forward wine.

6. **Syrah/Shiraz**: Depending on the region, Syrah or Shiraz can offer a range of flavors from peppery to fruity. Its boldness makes it a good companion to steak, especially when the steak is grilled or seasoned with bold spices.

Each of these wines brings its own unique characteristics to the dining experience, so the best choice may depend on your personal taste preferences and the specific cut of steak you are enjoying.

Exception in callback PollerCompletionQueue._handle_events(<_WindowsSele...e debug=False>)()
handle: <Handle PollerCompletionQueue._handle_events(<_WindowsSele...e debug=False>)()>
Traceback (most recent call last):
  File "c:\winapp\conda\envs\mas\Lib\asyncio\events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "src\\python\\grpcio\\grpc\\_cython\\_cygrpc/aio/completion_queue.pyx.pxi", line 147, in grpc._cython.cygrpc.PollerCompletionQueue._handle_events
BlockingIOError: [WinError 10035] A non-blocking socket operation could not be completed immediately
Exception in callback PollerCompletionQueue._handle_events(<_WindowsSele...e debug=False>)()
handle: <Handle PollerCompletionQueue._handle_events(<_WindowsSele...e debug=False>)()>
Traceback (most recent call last):
  File "c:\winapp\conda\envs\mas\Lib\asyncio\events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "src\\python\\grpcio\\grpc\\_cython\\_cygrpc/aio/completi

### Assistant API

Now, to support the conversation history, we need to use AI Assistant API. `thread` object is used to keep history, and `assistant` object holds together base language model and tools. Here is how we can create a thread and an assistant:

In [3]:
expiration_policy = { "ttl_days" : 1, "expiration_policy" : "static" }

def create_thread():
    return sdk.threads.create(**expiration_policy)

def create_assistant(model, tools=None):
    kwargs = expiration_policy.copy()
    if tools and len(tools) > 0:
        kwargs = {"tools": tools}
    return sdk.assistants.create(
        model, **kwargs
    )

In [4]:
thread = create_thread()
assistant = create_assistant(model)

assistant.update(
    instruction="You are experiences sommelier, and your task is consult a user about food-wine pairing"
)

thread.write("Hi! Which wine do you recommend?")

run = assistant.run(thread)
result = run.wait()
printx(result.text)

Hello! To provide the best recommendation, could you please tell me what type of dish you're planning to pair the wine with? Are you having a specific cuisine in mind, such as Italian, French, or Asian? And what kind of flavors are you working with—light and fresh, rich and hearty, spicy, or something else?

In [5]:
thread.write("I want to eat steak!")

run = assistant.run(thread)
result = run.wait()
printx(result.text)

For steak, I recommend a full-bodied red wine with good structure and moderate tannins to complement the richness and depth of flavor. A Cabernet Sauvignon or a Syrah/Shiraz would be excellent choices. These wines often have the right balance of fruitiness and robustness to pair well with the bold flavors of a steak. If you prefer a bit more complexity, a Merlot or a blend like a Bordeaux could also be a great option. What type of steak are you planning to cook, and do you have any specific flavor profiles in mind?

Thread now contains full history of messages:

In [6]:
for msg in list(thread)[::-1]:
    printx(f"**{msg.author.role}:** {msg.text}")

**USER:** Hi! Which wine do you recommend?

**ASSISTANT:** Hello! To provide the best recommendation, could you please tell me what type of dish you're planning to pair the wine with? Are you having a specific cuisine in mind, such as Italian, French, or Asian? And what kind of flavors are you working with—light and fresh, rich and hearty, spicy, or something else?

**USER:** I want to eat steak!

**ASSISTANT:** For steak, I recommend a full-bodied red wine with good structure and moderate tannins to complement the richness and depth of flavor. A Cabernet Sauvignon or a Syrah/Shiraz would be excellent choices. These wines often have the right balance of fruitiness and robustness to pair well with the bold flavors of a steak. If you prefer a bit more complexity, a Merlot or a blend like a Bordeaux could also be a great option. What type of steak are you planning to cook, and do you have any specific flavor profiles in mind?

After having used the assistant, we need to clean up:

In [7]:
thread.delete()
assistant.delete()

### Adding RAG content

Assistant API support easily adding RAG content to the assistant. To do that, we need to:
* Upload content to the cloud, either using `sdk.files.upload` or `sdk.files.upload_bytes`. API supports files of many common types, including PDF and DOCX, it automatically extracts text content from them.
* Create search index, specifying:
   - Index type: full-text (keyword) search, vector (semantic) search, or combined index (in which case we provide combination strategy)
   - Chunking strategy for splitting large files into smaller chunks
* Specify search tool when creating an assistant.

We will enhance our agent with food-wine matching capabilities using the table of this kind:

In [8]:
with open("../data/food_wine_table_en.md", encoding="utf-8") as f:
    food_wine = f.readlines()
fw = "".join(food_wine)
printx(fw[:1000])

| Dish to Pair with Wine | Recommended Wine |
|-----------------------|------------------|
| Eggplant baked with cheese | Red wine: Medium-bodied* dry wines—Grenache (Garnacha), Sangiovese (Chianti), Carmenère, Mencía, young Tempranillo, light-bodied Merlot. |
| Delicate lamb (lamb fillet or rack) | Red wine: Aged dry wines from Pinot Noir, Mencía, Nebbiolo (including elegant aged Barolo and Barbaresco), Gamay (elegant Burgundy Beaujolais-Villages). |
| Spicy lamb: grilled, roasted, or stewed with spices | Red wine: Dry wines from Cabernet Sauvignon, Rhône-style** blends (Grenache + Syrah + Mourvèdre), French Malbec, slightly rounded Barbera, Syrah (Shiraz). Aged Sangiovese (Chianti Classico, Montalcino wines), Aglianico, "Super Tuscan"*** wines, quality Rioja Crianza. Primitivo and Zinfandel. Russian Saperavi. |
| Beef Stroganoff | White wine: Oak-aged Chardonnay, Pinot Grigio (preferably from Northern Italy), Verdejo, Vermentino. Not overly powerful or acidic (and affordable) Rieslin

For table like this, it would be best to use manual chunking strategy, and keep table header present in each chunk to preserve semantic meaning of columns. Here we chunk table into pieces and upload them to the cloud:

In [9]:
header = food_wine[:2]
chunk_size = 2000 # chars

s = header.copy()
uploaded = []
for x in food_wine[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        id = sdk.files.upload_bytes(
            "".join(s).encode(),
            mime_type="text/markdown",
            **expiration_policy
        )
        uploaded.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded)} table chunks")

Uploaded 15 table chunks


Now we will create index from those chunks:

In [10]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy,
)

op = sdk.search_indexes.create_deferred(
    uploaded,
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
index = op.wait()

And now we can create an agent that uses this index:

In [11]:
search_tool = sdk.tools.search_index(index)
assistant = create_assistant(model, tools=[search_tool])

instruction = """
You are experiences sommelier, and your task is consult a user about food-wine pairing.
Please have a look at the available information and answer users's question as good as you can.
"""

_ = assistant.update(instruction=instruction)

In [12]:
thread = create_thread()
thread.write("Hi! Which wine is best with steak?")

run = assistant.run(thread)
result = run.wait()
printx(result.text)

The best wine pairing for steak depends on the type of steak and its level of doneness.

For marbled fatty beef steaks like ribeye:
- If rare, consider aged "noble" Tempranillo (Ribera del Duero Reserva+), Sangiovese (Chianti Riserva, Brunello), "Super Tuscan," Right Bank Bordeaux, silky Argentine Malbec.
- If medium/well-done, try dry/off-dry Syrah (Shiraz), Cabernet Sauvignon, full-bodied Malbec, Primitivo, Zinfandel, aged Aglianico, aged Rhône blends (Grenache + Syrah + Mourvèdre), or Priorat (6–8+ years aging).

For marbled beef tenderloin (filet mignon), lighter options like Pinot Noir, Nerello Mascalese, elegantly made and aged Merlot, rounded aged Nebbiolo (Barbaresco), Tempranillo (Ribera del Duero), or Sangiovese (Chianti Riserva) are recommended.

We can also find out which text fragments were used when answering this question:

In [13]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != "filechunk":
                continue
            print("------------------------")
            printx(source.parts[0])

print_citations(result)

------------------------


| Dish to Pair with Wine | Recommended Wine |
|-----------------------|------------------|
| Fried asparagus | White wine: Still dry Chardonnay, Grüner Veltliner, Müller-Thurgau, Sauvignon Blanc, light low-acid Riesling, Pinot Gris. Alternatively, sparkling (full-bodied, slightly off-dry—Brut, Dry, Off-Dry). |
| Marbled beef tenderloin (filet mignon) | Red wine: Light and elegant Pinot Noir, Nerello Mascalese, elegantly made and aged Merlot. Also, rounded aged Nebbiolo (Barbaresco), Tempranillo (Ribera del Duero), Sangiovese (Chianti Riserva). |
| Marbled fatty beef steak (ribeye, etc.) | Red wine: For rare—aged "noble" Tempranillo (Ribera del Duero Reserva+), Sangiovese (Chianti Riserva, Brunello), "Super Tuscan," Right Bank Bordeaux, silky Argentine Malbec. For medium/well-done—dry/off-dry Syrah (Shiraz), Cabernet Sauvignon, full-bodied Malbec, Primitivo, Zinfandel, aged Aglianico, aged Rhône blends (Grenache + Syrah + Mourvèdre), Priorat (6–8+ years aging). |
| Sushi, sashimi | (See "Classic sushi rolls") |
| Young goat cheese | White wine: Still dry—Old World Sauvignon Blanc (Pouilly-Fumé, Sancerre), Melon de Bourgogne (French Muscadet), Chardonnay (Chablis style). Alternatively, sparkling rosé (Extra Brut, Brut, Dry). |
| Aged goat cheese | White wine: Off-dry Riesling, aged "creamy" Chardonnay, classic Gewürztraminer or Muscat. Also, aged Bordeaux Sémillon and oaked Sauvignon Blanc (Sauternes). |
| Soft white-mold cheese (Brie, Camembert) | Sparkling wine: White (Brut Nature, Extra Brut, Brut), elegant rosé (Extra Brut, Brut). Also, still white—light elegant Chardonnay, Pinot Gris, Viura... Alternatively, refined still rosé from Provence (France). |
| Soft blue-mold cheese (Dor Blue, etc.) | White wine: Off-dry and sweet Gewürztraminer and Muscat, off-dry Riesling (Auslese, Beerenauslese, Eiswein), dense oaked Sémillon, aged Sauvignon Blanc, sweet white Port or dessert wines. |
| Semi-hard sweetish cheese (Maasdam, Gouda, Edam, etc.) | Red wine: Still dry—Bordeaux blends, Rhône blends (Grenache + Syrah + Mourvèdre), Cabernet Sauvignon, Sangiovese, Nero d’Avola, Tempranillo. Alternatively, full-bodied Pinot Noir, Gamay (Beaujolais-Villages). |

------------------------


| Dish to Pair with Wine | Recommended Wine |
|-----------------------|------------------|
| Chicken Caesar salad | White wine: Dry and off-dry Chardonnay, Verdejo, Pinot Gris, Chenin Blanc. Also, light elegant Provençal or Northern Italian rosé. Light orange wines work (especially if chicken is grilled). |
| Salmon Caesar salad | White wine: Dry aged Chardonnay, Albariño, Pinot Gris (Alto Adige). Off-dry whites—Sémillon, Riesling, Verdejo. Rosé wine: Light elegant from Provence (France) or Northern Italy. |
| Spicy Asian salads | White wine: Dry and off-dry Riesling, Muscat, Gewürztraminer. Rosé wine: Medium- to full-bodied from France, Italy, Spain, Russia. Alternatively, affordable full-bodied sparkling rosé. For salads with smoked/spicy meat—light reds: Pinot Noir, Nerello Mascalese, aged Nebbiolo. Orange wines work. |
| Salted or smoked lard | Strong drinks: Samogon (Polugar, Khlebnoe Vino), Vodka, Khrenovukha, Pepperspirit. |
| Pork in sweet-sour sauce | White wine: Riesling, dry Gewürztraminer or similar Muscat. |
| Lean pork medallions | White wine: Still dry aged Chardonnay, Albariño, Pinot Gris (Alto Adige). Off-dry whites—Sémillon, Riesling, Verdejo. Rosé wine: Medium- to full-bodied from France, Italy, Spain, Russia. |
| Fried pork steak with onions | Red wine: Dry and off-dry Garnacha (Grenache), Merlot, Carmenère, Mencía, full-bodied Pinot Noir (worldwide), Russian Krasnostop, Gamay (Beaujolais-Villages). |
| Herring with onions (appetizer) | Strong drinks: Samogon (Polugar, Khlebnoe Vino), Vodka, Khrenovukha, Pepperspirit. For wine purists—bold, off-dry, full-flavored Riesling. |
| Solyanka (soup) | Strong drinks: Samogon (Polugar, Khlebnoe Vino), Vodka, Khrenovukha, Pepperspirit. |
| Spaghetti Carbonara | White wine: Dry aged Chardonnay and Verdejo. Rosé wine: Medium- to full-bodied from France, Italy, Spain, Russia. Red wine: Aged rounded Nebbiolo, elegant Pinot Noir, Nerello Mascalese. |
| Spaghetti Bolognese | Red wine: Dry Sangiovese, Barbera, Nero d’Avola, Negroamaro, young Nebbiolo, also international—Cabernet Sauvignon, young Spanish Tempranillo. |

------------------------


| Dish to Pair with Wine | Recommended Wine |
|-----------------------|------------------|
| Delicate mushrooms (champignons, chanterelles, etc.) | White wine: Dry and off-dry aged Chardonnay, Viognier, dry Gewürztraminer (for chanterelles or cream-based champignons). Rosé wine: Still or sparkling (especially from Pinot Noir). |
| Mushroom julienne | Red wine: Dry Pinot Noir, Mencía, Nerello Mascalese, aged and velvety Nebbiolo. Rosé wine: Medium- to full-bodied Grenache, Cinsault, Tempranillo, Bobal, Syrah, or full-bodied sparkling rosé. |
| Jellied white fish | Strong drinks: Vodka, Polugar, Khrenovukha. Alternatively, white wine: Albariño, Verdejo, Sauvignon Blanc, Müller-Thurgau, Riesling. |
| Red caviar on crackers or white bread | White wine: Still, slightly sweet, and aged Pinot Grigio, Pinot Blanc, Chenin Blanc, Viognier, Sémillon, Garganega (Soave). Refined Chardonnay (Chablis style). |
| Black caviar on crackers or white bread | Despite common advice, pairing black caviar with still or sparkling wine is not recommended. However, if you insist, first take a sip of vodka or another non-oak-aged distillate, then enjoy the caviar plain or on bread. |
| Turkey (steaks, cutlets, skewers) | White wine: Dry and off-dry aged Chardonnay, Verdejo, Vermentino, Sémillon, Pinot Gris, Chenin Blanc. Medium-bodied rosé or orange wines also work. |
| Lamb rack | Red wine: Medium-bodied wines with balanced acidity—well-aged "Super Tuscan" wines, Brunello di Montalcino, Chianti Riserva, rounded and juicy Nero d’Avola, not-too-heavy Syrah (Shiraz), Rioja Reserva, well-made Garnacha and Barbera, full-bodied Pinot Noir (New World, Austria, Germany), mature and balanced Bordeaux and Piedmont wines (including Barolo and Barbaresco), aged Areni (Armenia). |
| Fried or French fries | White wine: Aged Verdejo, Vermentino, Pinot Gris, Grüner Veltliner, Silvaner, Sémillon, Chardonnay. Alternatively, light beer. |
| Quiche with various fillings | See "Pie" section. |
| Grilled sausages (spicy) | Beer: Light or dark, dense—lagers or ales. Alternatively, affordable young red wines: Sangiovese (Chianti), Nero d’Avola, Tempranillo (Spain), Merlot (worldwide), Blaufränkisch, Tsimlyansky Black, Russian Saperavi, Cabernet Sauvignon. Affordable Bordeaux blends. |

------------------------


| Dish to Pair with Wine | Recommended Wine |
|-----------------------|------------------|
| Eggplant baked with cheese | Red wine: Medium-bodied* dry wines—Grenache (Garnacha), Sangiovese (Chianti), Carmenère, Mencía, young Tempranillo, light-bodied Merlot. |
| Delicate lamb (lamb fillet or rack) | Red wine: Aged dry wines from Pinot Noir, Mencía, Nebbiolo (including elegant aged Barolo and Barbaresco), Gamay (elegant Burgundy Beaujolais-Villages). |
| Spicy lamb: grilled, roasted, or stewed with spices | Red wine: Dry wines from Cabernet Sauvignon, Rhône-style** blends (Grenache + Syrah + Mourvèdre), French Malbec, slightly rounded Barbera, Syrah (Shiraz). Aged Sangiovese (Chianti Classico, Montalcino wines), Aglianico, "Super Tuscan"*** wines, quality Rioja Crianza. Primitivo and Zinfandel. Russian Saperavi. |
| Beef Stroganoff | White wine: Oak-aged Chardonnay, Pinot Grigio (preferably from Northern Italy), Verdejo, Vermentino. Not overly powerful or acidic (and affordable) Rieslings. |
| Pancakes with beef filling | Strong drinks: Vodka, Polugar, Khrenovukha. Also, red wine: Italian Sangiovese (Chianti), Bordeaux blends, Rhône-style blends (Grenache + Syrah + Mourvèdre). |
| Pancakes with chicken filling | Strong drinks: Vodka, Polugar, Khrenovukha. Alternatively, white wine: Chardonnay, Verdejo, Vermentino, Trebbiano, Pinot Gris, dry Sherry. |
| Pancakes with red fish or caviar | Strong drinks: Vodka, Polugar, Khrenovukha. White wine: Riesling, Albariño, dry Muscat or Gewürztraminer, dry Sherry. |
| Borscht | Strong drinks: Samogon (Polugar, Khlebnoe Vino), Vodka, Khrenovukha. |
| Roasted or stewed beef (excluding marbled steaks) | Red wine: Dry Sangiovese (Chianti) and Nero d’Avola, Spanish Tempranillo, full-bodied Merlot, full-bodied Pinot Noir (New World, Austria, Germany), Bordeaux wines, Rhône blends (Grenache + Syrah + Mourvèdre), Saperavi (Georgia or Russia), Areni (Armenia). |
| Forest mushrooms (noble varieties)—fried or stewed | For medium-flavored: White wine: Dry and off-dry aged Chardonnay, Viognier, dry Gewürztraminer. For spicier: Red wine: Dry Pinot Noir, Nerello Mascalese, aged and velvety Nebbiolo. |

------------------------


| Dish to Pair with Wine | Recommended Wine |
|-----------------------|------------------|
| Roasted pork knuckle | Beer: German and Czech classic light lagers. Alternatively, white wine: Dry and off-dry Riesling, Silvaner, Müller-Thurgau, Grüner Veltliner, unoaked Chardonnay, etc. Don’t forget horseradish, mustard, and dumplings!!! |
| Mild white sea fish (pollock, cod, sea bass, halibut, catfish, etc.) | White wine: Still, light—Pinot Blanc, Pinot Gris, Viura, Melon de Bourgogne (Muscadet), Aligoté, Cortese (Gavi). Refined Chardonnay (Chablis style). Slightly sparkling Vinho Verde. Alternatively, velvety sparkling whites (Blanc de Blancs style). |
| Medium-flavored white sea fish (dorado, flounder, sea bass, etc.) | White wine: Bright but unoaked Albariño, Chardonnay, Riesling, Sauvignon Blanc, Cortese (Gavi), Silvaner, Müller-Thurgau, Grüner Veltliner, off-dry Riesling. Alternatively, sparkling (bright young whites, very light rosé). |
| Grilled spicy white fish (Asian-style) | White wine: Vibrant with higher acidity—Albariño, Riesling, Sauvignon Blanc, dry/off-dry Gewürztraminer, Muscat, Torrontés. Rosé wine: Medium-bodied dry from France, Italy, Spain, Russia. |
| Bold-flavored sea fish (tuna, mackerel, sardines) | White wine: Still—dry/off-dry vibrant Riesling, aged Chardonnay and Viognier, New World Sauvignon Blanc. Rosé wine: Medium-bodied dry from France, Italy, Spain, Russia. Alternatively, light reds—Pinot Noir, Nerello Mascalese, even aged Gamay (Beaujolais-Villages). |
| Mild river fish (pike perch, pike, asp, silver carp, carp) | White wine: Still dry Old World Sauvignon Blanc, Silvaner, Chenin Blanc, Müller-Thurgau, Grüner Veltliner. Full-bodied Pinot Gris, off-dry Riesling. Alternatively, sparkling (bright young whites, light rosé). |
| Dried fish | Beer: Classic light lagers or not-too-bold light ales. |
| Smoked sea fish (mackerel, saury) | Beer: Bold light or dark lagers (Germany, Czech Republic). Alternatively, white wine: Off-dry high-acid Riesling. |

In [14]:
thread.delete()
assistant.delete()

### Function Calling

Suppose we want to turn this assistant into a selling agent for a wine shop. In this case, we need it to be able to find specific wines from our inventory. First, let's load the sample inventory:

In [17]:
import pandas as pd
df = pd.read_csv("../data/wine-price-en.csv")
df

,Name,Price,Color,Acidity,Country
0,Robert Mondavi Cabernet Sauvignon 2019,32.99,Red,Dry,USA
1,Kendall-Jackson Vintner’s Reserve Chardonnay 2021,17.99,White,Dry,USA
2,Chateau Ste. Michelle Riesling 2020,10.99,White,Semi-Sweet,USA
3,Silver Oak Napa Valley Cabernet Sauvignon 2018,129.00,Red,Dry,USA
4,Duckhorn Merlot 2020,55.00,Red,Dry,USA
...,...,...,...,...,...
82,Château Cheval Blanc 2019,750.00,Red,Dry,France
83,Château Haut-Brion 2019,750.00,Red,Dry,France
84,Château d'Yquem 2018,750.00,White,Sweet,France
85,Château d'Agapet 2019,750.00,Red,Dry,France


To allow user to look up available wines, using RAG is not enough. We need to use **function calling**, which will give required flexibility to answer user queries. This can be done in two ways:
* Using a model to formulate user's request using SQL or SQL-like language
* Extracting some structured user query in JSON, and then running it manually on the given data, as described in [Querying Databases with Function Calling](https://arxiv.org/html/2502.00032v1)
We will try second approach.

We will inform our assistant that is has the available tool, and give it a class that describes tool parameters:

In [18]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
    """This function allows us to search wines in price list by several parameters"""

    name: str = Field(description="Name of wine", default=None)
    country: str = Field(description="Country", default=None)
    acidity: str = Field(
        description="Acidity (dry, semi-dry, sweet)", default=None
    )
    color: str = Field(description="Color (red, white)", default=None)
    output: str = Field(
        description="Output format (most expensive, cheapest, random, average, all)",
        default=None,
    )

price_list_search_tool = sdk.tools.function(SearchWinePriceList)

assistant = create_assistant(model, tools=[price_list_search_tool, search_tool])
thread = create_thread()

instruction = """
You are an experienced sommelier who acts as a seller to a wine shop.
You have to answer user's questions on wines, recommend best food for wine and vice versa,
and search wines in our price list. Look at the available information before answering.
If the question is about specific wines, wine availability or wine pricing - use
function calling. If something is not clear - please, ask the user.
"""

_ = assistant.update(instruction=instruction)

In [19]:
thread.write("Hey! Which is the cheapest wine from Australia?")
run = assistant.run(thread)
res = run.wait()
res

RunResult(status=<RunStatus.TOOL_CALLS: 5>, error=None, tool_calls=ToolCallList(ToolCall(function=FunctionCall(name='SearchWinePriceList', arguments={'output': 'cheapest', 'country': 'Australia'})),), _message=None, usage=Usage(input_text_tokens=3515, completion_tokens=18, total_tokens=3533))

As you can see, the assistant now wants to run a function call. In order to do it, we first need to define a function to query our price list:

In [23]:
def find_wines(req):
    x = df.copy()
    if req.country:
        x = x[x["Country"] == req.country]
    if req.acidity:
        x = x[x["Acidity"] == req.acidity.capitalize()]
    if req.color:
        x = x[x["Color"] == req.color.capitalize()]
    if req.name:
        x = x[x["Name"].apply(lambda x: req.name.lower() in x.lower())]
    if req.output and len(x)>0:
        if req.output == "cheapest":
            x = x.sort_values(by="Price")
        elif req.output == "most expensive":
            x = x.sort_values(by="Price", ascending=False)
        else:
            pass
    if x is None or len(x) == 0:
        return "Wines not found"
    return "The following wines were found:\n" + "\n".join(
        [
            f"{z['Name']} (z['Country']) - {z['Price']}"
            for _, z in x.head(10).iterrows()
        ]
    )

Now we will run the request again, this time processing function calling response:

In [26]:
import time

thread.delete()
thread = create_thread()

thread.write("Hey! Which is the cheapest wine from Australia?")

run = assistant.run(thread)
res = run.wait()
if res.tool_calls:
    result = []
    for f in res.tool_calls:
        print(f" + Processing function call fn={f.function.name}")
        x = SearchWinePriceList.model_validate(f.function.arguments)
        x = find_wines(x)
        result.append({"name": f.function.name, "content": x})
    run.submit_tool_results(result)
    time.sleep(3)
    res = run.wait()
res

 + Processing function call fn=SearchWinePriceList


RunResult(status=<RunStatus.COMPLETED: 4>, error=None, tool_calls=None, _message=Message(id='fvt9h10tvhrtr1elk2nu', parts=("The cheapest wine from Australia is Lindeman's Bin 65 Chardonnay 2022, priced at $6.99.",), thread_id='fvte6jrbb2er5og6206i', created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 7, 16, 8, 50, 31, 54000), labels=None, author=Author(id='fvt7hv5a4ab1dtu1vrd1', role='ASSISTANT'), citations=(), status=<MessageStatus.COMPLETED: 1>), usage=Usage(input_text_tokens=3949, completion_tokens=49, total_tokens=3998))

In [27]:
thread.delete()
assistant.delete()

### Putting Everything Together

We will now create an agent class that will encapsulate RAG search and function calling.

In [58]:
class Agent:
    def __init__(self, 
        assistant=None, 
        instruction=None, 
        search_index=None, 
        search_content=None, 
        tools=None,
        response_format=None):

        self.thread = None
        self.uploaded = None
        self.model = model
        self.response_format = response_format

        if assistant:
            self.assistant = assistant
        else:
            if tools:
                self.tools = {x.__name__: x for x in tools}
                tools = [sdk.tools.function(x) for x in tools]
            else:
                self.tools = {}
                tools = []
            if search_index is None and search_content:
                self.uploaded, search_index = self.create_search_index(search_content)
            if search_index:
                tools.append(sdk.tools.search_index(search_index))
            if response_format:
                self.model = self.model.configure(response_format=response_format)
            self.assistant = create_assistant(self.model, tools)
            self.search_index = search_index

        if instruction:
            self.assistant.update(instruction=instruction)

    def create_search_index(self,content):
        uploaded = [
            sdk.files.upload_bytes(
                x.encode(), 
                ttl_days=5, 
                expiration_policy="static",
                mime_type="text/markdown")
            for x in content]

        op = sdk.search_indexes.create_deferred(
            uploaded,
            index_type=HybridSearchIndexType(
                chunking_strategy=StaticIndexChunkingStrategy(
                    max_chunk_size_tokens=1000, chunk_overlap_tokens=100
                ),
                combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
            ),
        )
        return uploaded, op.wait()

    def get_thread(self, thread=None):
        if thread is not None:
            return thread
        if self.thread == None:
            self.thread = create_thread()
        return self.thread

    def __call__(self, message, thread=None):
        thread = self.get_thread(thread)
        thread.write(message)
        run = self.assistant.run(thread)
        res = run.wait()
        if res.tool_calls:
            result = []
            for f in res.tool_calls:
                print(
                    f" + Calling function {f.function.name}, args={f.function.arguments}"
                )
                fn = self.tools[f.function.name]
                obj = fn(**f.function.arguments)
                x = obj.process(thread)
                result.append({"name": f.function.name, "arguments" : f.function.arguments, "content": x})
            run.submit_tool_results(result)
            res = run.wait()
        res = res.text
        if self.response_format:
            res = self.response_format.model_validate_json(res[res.index("{"):res.index("}")+1])
        return res 

    def restart(self):
        if self.thread:
            self.thread.delete()
            self.thread = sdk.threads.create(
                name="Test", ttl_days=1, expiration_policy="static"
            )

    def done(self, delete_assistant=False):
        if self.search_index:
            self.search_index.delete()
        if self.thread:
            self.thread.delete()
        if delete_assistant:
            self.assistant.delete()

Let's now use this to define and agent for the shop that allows transferring to operator, and buyer agent, and have them talk to each other. We first define tools:

In [59]:
class SearchWinePriceList(BaseModel):
    """This function allows us to search wines in price list by several parameters"""

    name: str = Field(description="Name of wine", default=None)
    country: str = Field(description="Country", default=None)
    acidity: str = Field(
        description="Acidity (dry, semi-dry, sweet)", default=None
    )
    color: str = Field(description="Color (red, white)", default=None)
    output: str = Field(
        description="Output format (most expensive, cheapest, random, average, all)",
        default=None,
    )

    def process(self,thread):
        return find_wines(self)

handover = False

class Handover(BaseModel):
    """This function allows to handover the conversation to human operator"""

    reason: str = Field(
        description="Reason for calling the operator", default="not specified"
    )

    def process(self, thread):
        global handover
        handover = True
        return f"I am calling operator on {thread.id=}, reason: {self.reason}"

Now creating the main agent:

In [60]:
with open("../data/food_wine_table_en.md", encoding="utf-8") as f:
    food_wine = f.readlines()
header = food_wine[:2]
chunk_size = 2000
docs = []
s = header.copy()
for x in food_wine[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        docs.append("".join(s))
        s = header.copy()

instruction = """
You are an experienced sommelier who acts as a seller to a wine shop.
You have to answer user's questions on wines, recommend best food for wine and vice versa,
and search wines in our price list. Look at the available information before answering.
If the question is about specific wines, wine availability or wine pricing - use
function calling. If something is not clear - please, ask the user.
"""

agent = Agent(search_content=docs, instruction=instruction, tools=[SearchWinePriceList,Handover])

And now we will create a buyer agent and give it some instructions on what to do:

In [61]:
buyer_inst = """
You are a normal person, who came to wine shop to buy some wine for dinner. You do not know much about
wines, so you want to ask seller for recommendation. You need not very expensive wine, so ask for the price before final decision.
When you are satisfied - call the operator to 
process the order.
"""
buyer = Agent(instruction=buyer_inst)

In [62]:
msg = "Hello!"
handover = False
for i in range(10):
    printx(f"**Seller:** {msg}")
    msg = buyer(msg)
    printx(f"**Buyer:** {msg}")
    msg = agent(msg)
    if handover:
        break
    

**Seller:** Hello!

**Buyer:** Hello! I'm here to pick up some wine for dinner, but I don't know much about wines. Could you recommend a good one that's not too expensive?

**Seller:** Of course, I'd be happy to help! To give you the best recommendation, could you please tell me what kind of dish you plan to have with the wine? This will help me suggest a wine that pairs well with your meal.

**Buyer:** I'm planning to have a steak for dinner. What kind of wine would you recommend to go with that? And could you also let me know the price range of your suggestions?

 + Calling function SearchWinePriceList, args={'color': 'red', 'output': 'cheapest'}


**Seller:** For a steak dinner, I would recommend a red wine to complement the flavors. Here are some affordable options from our selection:

1. Yellow Tail Shiraz 2022 - $7.49
2. Mirassou Pinot Noir 2021 - $9.99
3. Line 39 Cabernet Sauvignon 2021 - $9.99
4. Menage à Trois Red Blend 2021 - $9.99
5. 19 Crimes Red Blend 2021 - $9.99
6. Bodegas Borsao Garnacha 2021 - $9.99
7. La Vieille Ferme Rouge 2021 - $9.99
8. Alamos Malbec 2021 - $10.49
9. Gnarly Head Old Vine Zinfandel 2021 - $10.99
10. Mark West Pinot Noir 2021 - $10.99

These wines should pair nicely with your steak and are within a reasonable price range. Let me know if you need more information or if there's a specific wine you'd like to know more about!

**Buyer:** Thank you for the recommendations! The prices are reasonable, and I think I'll go with the Alamos Malbec 2021. Could you please call the operator to process my order?

 + Calling function Handover, args={'reason': 'To process the order for Alamos Malbec 2021'}


In [63]:
agent.done()
buyer.done()

## LangGraph Agentic Workflow

Now let's consider more complex problem - selecting food-wine pairing for dinner for a person with some initial preferences. In order to do this, we will simulate the real-life process of visiting a restaurant, which includes the following agents:
* Hostess to find out initial preferences of a visitor
* Waiter, who knows the menu of the restaurant and can answer corresponding questions
* Sommelier, who knows the food-wine pairing

#### The Waiter

We will create the waited based on RAG for simplicify, but if the menu is big - you should probably be using function calling...

In [65]:
prompt = """
You are a waiter in a restaurant, whose goal is to recommend food to visitors
and wo answer questions on menu. You have extracts from the menu at your disposal.
Please answer using only available information, do not invent anything. 
"""

def fread(fn):
    with open(fn, encoding='utf-8') as f:
        return f.read()

waiter = Agent(
    search_content=[fread('../data/menu/food_en.md'),fread('../data/menu/drinks_en.md')],
    instruction=prompt)  

In [66]:
printx(waiter("How much is ice-cream?"))

The menu does not list the price for ice cream separately. However, there is a dessert called "Chocolate Apocalypse" which includes "Heavenly Cloud" ice cream, and its price is 800 RUB.

In [67]:
printx(waiter("Which is the most expensive wine?"))

The most expensive wine is Pinot Noir from Domaine de la Romanée-Conti, France, priced at 5200 RUB per glass.

#### Sommelier

Sommelier agent will use the food-wine pairing table:

In [68]:
prompt = """
You are experienced sommelier, and you should recommend food-wine pairing to the user.
You have access to food-wine pairing table, which you should use to find the answer.
"""

sommelier = Agent(
    search_content=docs,
    instruction=prompt)  

In [69]:
printx(sommelier("Which wine would be good for steak?"))

The wine you choose for steak depends on the type of steak and its preparation. Here are some recommendations:

- For marbled beef tenderloin (filet mignon), consider red wine such as light and elegant Pinot Noir, Nerello Mascalese, elegantly made and aged Merlot. Also, rounded aged Nebbiolo (Barbaresco), Tempranillo (Ribera del Duero), Sangiovese (Chianti Riserva) would be a good choice.
- For marbled fatty beef steak (ribeye, etc.), aged "noble" Tempranillo (Ribera del Duero Reserva+), Sangiovese (Chianti Riserva, Brunello), "Super Tuscan," Right Bank Bordeaux, silky Argentine Malbec would be suitable for rare steak. For medium/well-done steak, dry/off-dry Syrah (Shiraz), Cabernet Sauvignon, full-bodied Malbec, Primitivo, Zinfandel, aged Aglianico, aged Rhône blends (Grenache + Syrah + Mourvèdre), Priorat (6–8+ years aging) are recommended.

In [70]:
printx(sommelier("Which wine would be good for filet mignon steak?"))

For filet mignon steak, consider red wine such as light and elegant Pinot Noir, Nerello Mascalese, elegantly made and aged Merlot. Also, rounded aged Nebbiolo (Barbaresco), Tempranillo (Ribera del Duero), Sangiovese (Chianti Riserva) would be a good choice.

#### Hostess Agent

In our case, the role of a hostess would be to find initial preferences of a user, in terms of food or wine. We will use **structured output** for this, and populate pydantic object:

In [73]:
from pydantic import BaseModel

class PersonPref(BaseModel):
    food_pref : str = ""
    wine_pref : str = ""
    
prompt = """
You are a hostess in a restaurant, and your goal is to figure out if a user has
some food preference (`food_pref`) or wine preference (`wine_pref`) for his meal.
If there are no preferences specified - return empty strings. Please return
JSON containing the fields specified above.
"""

hostess = Agent(
    instruction=prompt,
    response_format=PersonPref)

In [74]:
res = hostess("I want to eat something from meat")
res

PersonPref(food_pref='meat', wine_pref='')

### Putting Agents together: LangGraph

To make all agents work together on solving a problem, we can use LangGraph framework. Its main concept is the communication **graph**, and **problem state**, which is passed around the graph and which determines which agent will interact next.

Nodes of the graph are functions, and they return **state updates**. 

First, let's describe the class that would hold the state:

In [75]:
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
from yandex_cloud_ml_sdk._threads.thread import Thread

class RecommenderState(TypedDict):
    message : str
    thread : Thread
    foodpref : str = ""
    winepref : str = ""
    maindish : str
    wine : str
    answer : str
    
def pr(state: RecommenderState):
    return ', '.join([f"{k}={v}" for k,v in state.items() if k!="thread"])

Initial message from the user is in the field `message`, and final answer - in `answer`. We will hold the common state of conversaton as `Thread` object in the field with the same name. Initial food and wine preferences would be in `foodpref` and `winepref`, and selected dish and wine - in `maindish` and `wine`.

The logic of the workflow would be the following:
* First, we exctract initial preferences using `welcome` function and hostess agent
* The main dispatcher function of our workflow would be `clarify`. If it sees some preferences in `foodpref` or `winepref` - it selects corresponding dishes using waiter agent.
* After `clarify`, `route_user` function is called, which defines what to do next:
   - If both main course and wine are selected, we go to `check_combination`, where we additionally check food-wine pairing using sommelier agent. If the user asked for a steak with white wine - this will be handled in this case.
   - If some of the `winepref` or `foodpref` have not been filled yet - we call `recommend_food` or `recommend_wine`, to have sommelier take care of this
   - If the user does not have preferences, we call `select_random` to select random main dish for him, and then go to `recommend_wine`.

So, to define agentic workflow, we need:
* Define all node functions
* Add all of the to the graph using `add_node`
* Add all transitions to the graph using `add_edge` and `add_conditional_edges`.

In [92]:
from IPython.display import Image, display
from langchain_core.runnables.graph_mermaid import MermaidDrawMethod

def welcome_user(state: RecommenderState):
    print(f"> Welcome, state={pr(state)}")
    msg = state['message']
    print(f" + Initial message from user: {msg}")
    res = hostess(msg,state['thread'])
    print(f" + Preferences obtained: {res}")
    return {
        'foodpref' : res.food_pref,
        'winepref' : res.wine_pref
    }
    
def route_user(state: RecommenderState) -> str:
    if state.get('maindish') and state.get('wine'): return "Done"
    if state['foodpref'] == "" and state['winepref'] == "": return "None"
    elif state['foodpref'] == "" and state['winepref'] != "": return "WantWine"
    elif state['foodpref'] != "" and state['winepref'] == "": return "WantFood"
    else: return "Both"

def select_random(state: RecommenderState):
    print(f"> SelectRandom, state={pr(state)}")
    maindish = waiter(
        "Please recommend some good main course dish. Write only the name of the dish in the answer",
        state["thread"])
    print(f" + Random dish selected: {maindish}")
    return { "maindish" : maindish }
    
def recommend_food(state: RecommenderState):
    print(f"> Recommend Food, state={pr(state)}")
    wine = state.get("wine") or state.get("winepref")
    print(f" + Recommending food for {wine}")
    foodpref = sommelier(f"Which dishes would be good for wine {wine}?", state['thread'])
    print(f" + Recommendations: {foodpref}")
    return { "foodpref" : foodpref }

def recommend_wine(state: RecommenderState):
    print(f"> Recommend wine, state={pr(state)}")
    food = state.get('maindish') or state.get('foodpref')
    print(f" + Recommending wine for food {food}")
    winepref = sommelier(f"Which wines would be good for dish {food}?", state['thread'])
    print(f" + Recommendations: {winepref}")
    return { "winepref" : winepref }
    
def clarify(state: RecommenderState):
    print(f"> Clarify, state={pr(state)}")
    upd = {}
    if state.get("winepref") and not state.get('wine'):
        print(f" + Looking for wine in menu: {state['winepref']}")
        upd["wine"] = waiter(f"Which of the wines in the menu correspond to the following description: {state['winepref']}? Select only one wine from the menu",state['thread'])
        print(f" + Wine selected: {upd['wine']}")
    if state.get("foodpref") and not state.get('maindish'):
        print(f" + Looking for food in menu: {state['foodpref']}")
        upd["maindish"] = waiter(f"Which of the dishes in the menu correspond to the following description: {state['foodpref']}? Select only one dish from the menu.",state['thread'])
        print(f" + Selected dish: {upd['maindish']}")
    return upd

def check_combination(state: RecommenderState):
    print(f"> Check combination, state={pr(state)}")
    food = state["maindish"]
    wine = state["wine"]
    print(f" + Looking for paring between dish {food} and wine {wine}")
    res = sommelier(f"Is the main course {food} good with wine {wine}? Please answer YES or NO",state['thread'])
    if "yes" not in res.lower():
        ans = waiter(f"Guests want to order {state['maindish']} with wine {state['wine']}. Please write polite answer that those food and wine do not go well together, and suggest to order something else.",state['thread'])
    else:
        ans = waiter(f"Please offer guests the following main dish {state['maindish']} and wine {state['wine']}.",state['thread'])
    return { "answer" : ans}

def yes_or_no(state: RecommenderState):
    if state["maindish"] and state["wine"]:
        return "Yes"
    else:
        return "No"

recommender_graph = StateGraph(RecommenderState)
recommender_graph.add_node("Welcome", welcome_user)
recommender_graph.add_node("SelectRandom", select_random)
recommender_graph.add_node("RecommendFood", recommend_food)
recommender_graph.add_node("RecommendWine", recommend_wine)
recommender_graph.add_node("CheckCombination", check_combination)
recommender_graph.add_node("Clarify", clarify)

recommender_graph.add_edge(START, "Welcome")
recommender_graph.add_edge("Welcome", "Clarify")
recommender_graph.add_conditional_edges(
    "Clarify",
    route_user,
    {
        "None": "SelectRandom",
        "WantWine" : "RecommendFood",
        "WantFood" : "RecommendWine",
        "Both" : "CheckCombination",
        "Done" : "CheckCombination"
    })
recommender_graph.add_edge("SelectRandom", "RecommendWine")
recommender_graph.add_edge("RecommendFood", "Clarify")
recommender_graph.add_edge("RecommendWine", "Clarify")
recommender_graph.add_edge("CheckCombination", END)

compiled_graph = recommender_graph.compile()

# display(Image(compiled_graph.get_graph().draw_mermaid_png()))

Для того, чтобы "поговорить" с этой системой, дадим ей на вход начальное состояние:

In [93]:
res = compiled_graph.invoke(
    {
        "message" : "Hello! I want something from meat",
        "thread" : create_thread()
    }
)
res

> Welcome, state=message=Hello! I want something from meat
 + Initial message from user: Hello! I want something from meat
 + Preferences obtained: food_pref='meat' wine_pref=''
> Clarify, state=message=Hello! I want something from meat, foodpref=meat, winepref=
 + Looking for food in menu: meat
 + Selected dish: **"Bull on Edge" Steak**
> Recommend wine, state=message=Hello! I want something from meat, foodpref=meat, winepref=, maindish=**"Bull on Edge" Steak**
 + Recommending wine for food **"Bull on Edge" Steak**
 + Recommendations: Red wine: Dry Sangiovese (Chianti) and Nero d’Avola, Spanish Tempranillo, full-bodied Merlot, full-bodied Pinot Noir (New World, Austria, Germany), Bordeaux wines, Rhône blends (Grenache + Syrah + Mourvèdre), Saperavi (Georgia or Russia), Areni (Armenia).
> Clarify, state=message=Hello! I want something from meat, foodpref=meat, winepref=Red wine: Dry Sangiovese (Chianti) and Nero d’Avola, Spanish Tempranillo, full-bodied Merlot, full-bodied Pinot Noir (

{'message': 'Hello! I want something from meat',
 'thread': Thread(id='fvt9fvtp0qegjg5eu0ac', expiration_config=ExpirationConfig(ttl_days=1, expiration_policy=<ExpirationPolicy.STATIC: 1>), name=None, description=None, created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 7, 16, 10, 38, 3, 759282), updated_by='ajej20rll4tifkelclga', updated_at=datetime.datetime(2025, 7, 16, 10, 38, 3, 759282), expires_at=datetime.datetime(2025, 7, 17, 10, 38, 3, 759282), labels=None),
 'foodpref': 'meat',
 'winepref': 'Red wine: Dry Sangiovese (Chianti) and Nero d’Avola, Spanish Tempranillo, full-bodied Merlot, full-bodied Pinot Noir (New World, Austria, Germany), Bordeaux wines, Rhône blends (Grenache + Syrah + Mourvèdre), Saperavi (Georgia or Russia), Areni (Armenia).',
 'maindish': '**"Bull on Edge" Steak**',
 'wine': 'Merlot by Marchesi Antinori, Italy.',
 'answer': 'I recommend the **"Bull on Edge" Steak**, which is a juicy ribeye slow-cooked in the smoke of Argentine passion and s

In [94]:
printx(res['answer'])

I recommend the **"Bull on Edge" Steak**, which is a juicy ribeye slow-cooked in the smoke of Argentine passion and served with Himalayan gold salt. To complement this steak, I suggest pairing it with Merlot by Marchesi Antinori, Italy. The price for the steak is 2500 RUB, and the wine is available by the glass for 2800 RUB.

We can also see all agent conversations:

In [97]:
for msg in list(res['thread'])[::-1]:
    printx(f"**{msg.role}:** {msg.text}")

**USER:** Hello! I want something from meat

**ASSISTANT:** ```
{
  "food_pref": "meat",
  "wine_pref": ""
}
```

**USER:** Which of the dishes in the menu correspond to the following description: meat? Select only one dish from the menu.

**ASSISTANT:** **"Bull on Edge" Steak**

**USER:** Which wines would be good for dish **"Bull on Edge" Steak**?

...

**ASSISTANT:** Merlot by Marchesi Antinori, Italy.

**USER:** Is the main course **"Bull on Edge" Steak** good with wine Merlot by Marchesi Antinori, Italy.? Please answer YES or NO

**ASSISTANT:** YES.

**USER:** Please offer guests the following main dish **"Bull on Edge" Steak** and wine Merlot by Marchesi Antinori, Italy..

**ASSISTANT:** I recommend the **"Bull on Edge" Steak**, which is a juicy ribeye slow-cooked in the smoke of Argentine passion and served with Himalayan gold salt. To complement this steak, I suggest pairing it with Merlot by Marchesi Antinori, Italy. The price for the steak is 2500 RUB, and the wine is available by the glass for 2800 RUB.

What if the user wants a bad combination?

In [98]:
res = compiled_graph.invoke(
    {
        "message" : "I want some fish with merlot!",
        "thread" : create_thread()
    }
)
printx(res['answer'])

> Welcome, state=message=I want some fish with merlot!
 + Initial message from user: I want some fish with merlot!
 + Preferences obtained: food_pref='fish' wine_pref='merlot'
> Clarify, state=message=I want some fish with merlot!, foodpref=fish, winepref=merlot
 + Looking for wine in menu: merlot
 + Wine selected: Merlot, Marchesi Antinori, Italy, 2019.
 + Looking for food in menu: fish
 + Selected dish: "Salmon Dreaming of Norway"
> Check combination, state=message=I want some fish with merlot!, foodpref=fish, winepref=merlot, maindish="Salmon Dreaming of Norway", wine=Merlot, Marchesi Antinori, Italy, 2019.
 + Looking for paring between dish "Salmon Dreaming of Norway" and wine Merlot, Marchesi Antinori, Italy, 2019.


Thank you for your order. I'm sorry to inform you that the "Salmon Dreaming of Norway" does not pair well with the Merlot, Marchesi Antinori, Italy, 2019. May I suggest a different wine that would complement your dish better? Perhaps a white wine such as Sauvignon Blanc or Chardonnay might be a good choice. Would you like me to recommend a specific wine from our menu?

And now a case when the user does not have have specific preferences:

In [99]:
res = compiled_graph.invoke(
    {
        "message" : "I want something good to eat!",
        "thread" : create_thread()
    }
)
printx(res['answer'])

> Welcome, state=message=I want something good to eat!
 + Initial message from user: I want something good to eat!
 + Preferences obtained: food_pref='' wine_pref=''
> Clarify, state=message=I want something good to eat!, foodpref=, winepref=
> SelectRandom, state=message=I want something good to eat!, foodpref=, winepref=
 + Random dish selected: **"Bull on Edge" Steak**
> Recommend wine, state=message=I want something good to eat!, foodpref=, winepref=, maindish=**"Bull on Edge" Steak**
 + Recommending wine for food **"Bull on Edge" Steak**
 + Recommendations: Red wine: Dry Sangiovese (Chianti) and Nero d’Avola, Spanish Tempranillo, full-bodied Merlot, full-bodied Pinot Noir (New World, Austria, Germany), Bordeaux wines, Rhône blends (Grenache + Syrah + Mourvèdre), Saperavi (Georgia or Russia), Areni (Armenia).
> Clarify, state=message=I want something good to eat!, foodpref=, winepref=Red wine: Dry Sangiovese (Chianti) and Nero d’Avola, Spanish Tempranillo, full-bodied Merlot, full-

I recommend the **"Bull on Edge" Steak** for your main course. It's a juicy ribeye slow-cooked in the smoke of Argentine passion, served with Himalayan gold salt. 

For wine, I suggest the Merlot by Marchesi Antinori, Italy. It's a full-bodied red wine that pairs excellently with the steak. 

The **"Bull on Edge" Steak** is priced at 2500 RUB, and the Merlot by Marchesi Antinori is priced at 2800 RUB per glass.

## Testing: RAGAS

Testing RAG and Multi-Agent systems are tricky for several reasons:

* Not obvious how to estimate quality of natural-language answer
* Difficult to test multi-turn conversations
* Need to separately evaluate quality of retrieval and answer pipelines

To solve some of those problems, there are tools such as [RAGAS](https://ragas.io).

There are several approaches to testing:
* We manually create question-answer pairs
* We manually create a set of questions and then assess answer quality using "LLM as a Judge" or some set of defined criteria
* QA Dataset is automatically created from RAG database. RAGAS contains different powerful techniques for building questions, including knowledge graph extraction and transformation.

Let's start with the first approach:

In [101]:
import json

ds = json.load(open('../data/eval/evaluate_advisor_en.json','r'))
ds

[{'user_input': 'I want to eat something from meat',
  'reference': 'I recommend steak and some red wine, such as Malbec or Merlot.'},
 {'user_input': 'I prefer fish',
  'reference': 'I recommend some fish dish such as grilled salmon, and some dry white wine, such as Riesling or Sauvignon Blanc.'},
 {'user_input': 'I want to eat salmon with red wine',
  'reference': 'I am afraid we do not recommend to pair fish with red wine'}]

Now let's run our agentic workflow on each case, and save the resulting response in `response` field:

In [102]:
for i,x in enumerate(ds):
    print(f"==== RUNNING EXAMPLE {i+1}: {x['user_input']}")
    res = compiled_graph.invoke(
    {
        "message" : x['user_input'],
        "thread" : create_thread()
    })
    ds[i]['response'] = res['answer']

with open('../data/eval/evaluate_advisor_results_en.json','w') as f:
    json.dump(ds,f,indent=4, ensure_ascii=False)

==== RUNNING EXAMPLE 1: I want to eat something from meat
> Welcome, state=message=I want to eat something from meat
 + Initial message from user: I want to eat something from meat
 + Preferences obtained: food_pref='meat' wine_pref=''
> Clarify, state=message=I want to eat something from meat, foodpref=meat, winepref=
 + Looking for food in menu: meat
 + Selected dish: **"Bull on Edge" Steak**
> Recommend wine, state=message=I want to eat something from meat, foodpref=meat, winepref=, maindish=**"Bull on Edge" Steak**
 + Recommending wine for food **"Bull on Edge" Steak**
 + Recommendations: Red wine: Dry Sangiovese (Chianti) and Nero d’Avola, Spanish Tempranillo, full-bodied Merlot, full-bodied Pinot Noir (New World, Austria, Germany), Bordeaux wines, Rhône blends (Grenache + Syrah + Mourvèdre), Saperavi (Georgia or Russia), Areni (Armenia).
> Clarify, state=message=I want to eat something from meat, foodpref=meat, winepref=Red wine: Dry Sangiovese (Chianti) and Nero d’Avola, Spanish

To speed up the demo we will just load existing data. If funning on Windows, we should also restart the kernel to set up the right async strategy to avoid extra warning messages.

In [4]:
import os
import json

if os.name == 'nt':
    os.environ["GRPC_POLL_STRATEGY"] = "poll"
    import asyncio
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

from yandex_cloud_ml_sdk import YCloudML, AsyncYCloudML

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)
asdk = AsyncYCloudML(folder_id=folder_id, auth=api_key)

In [5]:
ds = json.load(open('../data/eval/evaluate_advisor_results_en.json','r'))
ds

[{'user_input': 'I want to eat something from meat',
  'reference': 'I recommend steak and some red wine, such as Malbec or Merlot.',
  'response': 'I recommend the **"Bull on Edge" Steak** as your main dish. It\'s a juicy ribeye slow-cooked in the smoke of Argentine passion and served with Himalayan gold salt. For wine, I suggest the Merlot by Marchesi Antinori, Italy. The steak is priced at 2500 RUB and the wine is 2800 RUB per glass.'},
 {'user_input': 'I prefer fish',
  'reference': 'I recommend some fish dish such as grilled salmon, and some dry white wine, such as Riesling or Sauvignon Blanc.',
  'response': 'I recommend the main dish **"Salmon Dreaming of Norway"** — Delicate fillet baked under a crust of "mysterious northern herbs" (dill). Served with lemon breeze. \n\nTo complement this dish, I suggest pairing it with Riesling from Dr. Loosen, Germany, 2019.'},
 {'user_input': 'I want to eat salmon with red wine',
  'reference': 'I am afraid we do not recommend to pair fish wi

 Now we need to eastimate the similarity of provided answer and expected answer. For that, we can use semantic embeddings. RAGAS allows us to use embeddings from LangChain library: 

In [ ]:
from ragas.dataset_schema import EvaluationDataset, SingleTurnSample
from ragas import evaluate
from ragas.metrics import answer_similarity
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

embeddings = YandexGPTEmbeddings(api_key=os.environ['api_key'], folder_id=os.environ['folder_id'])
embeddings = LangchainEmbeddingsWrapper(embeddings)

tests = EvaluationDataset([ SingleTurnSample(**x) for x in ds ])
evaluate(tests,[ answer_similarity ], embeddings=embeddings)

Evaluating: 100%|██████████| 3/3 [00:03<00:00,  1.24s/it]


{'semantic_similarity': 0.7588}

RAGAS uses so-called **metrics** to check different quality aspects of responses. We have been using `answer_similarity` metrics above, based on embeddings.

Another way is to use LLM as a Judge. There is a number of metrics with pre-defined prompts (eg, `AnswerAccuracy`), or we can define our own metrics by providing prompts expliciptly.

Let's make sure our recommendations follow simple rule "red wine goes with meat, white - with fish":

In [7]:
from ragas.metrics import AspectCritic, AnswerAccuracy
from ragas.llms import LangchainLLMWrapper
from yandex_cloud_ml_sdk import YCloudML, AsyncYCloudML
asdk = AsyncYCloudML(folder_id=folder_id, auth=api_key)

judge_llm = LangchainLLMWrapper(asdk.models.completions("yandexgpt", model_version="rc").langchain())

match_criteria = """
Does the recommended wine match the recommended main dish well? White wines go well with fish,
red wines - with meat dishes. If the food and wine do not match, it should be explicitly indicated
in the answer.
"""
wine_food_match = AspectCritic("wine_food_match",match_criteria,llm=judge_llm)
wine_food_match.async_mode=False
answer_accuracy = AnswerAccuracy(llm=judge_llm)
answer_accuracy.async_mode=False

evaluate(tests,[ wine_food_match, answer_accuracy ], llm=judge_llm)

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 6/6 [00:02<00:00,  2.10it/s]


{'wine_food_match': 1.0000, 'nv_accuracy': 0.6667}

Having set up those measurements, we can now perform some meaningful optimizations of both agentic workflow configuration. In the similar manner, RAGAS is well-suited to measure RAG performance, as it contains special methods to evaluate retrieval performance separately from answer formulation.

## Cleaning up

**WARNING:** The code below deletes all AI Assistant API objects in your folder. Use with caution!

In [8]:
from tqdm.auto import tqdm

for thread in tqdm(sdk.threads.list()):
    try:
        thread.delete()
    except:
        pass

for assistant in tqdm(sdk.assistants.list()):
    assistant.delete()

for index in tqdm(sdk.search_indexes.list()):
    index.delete()
    
for file in tqdm(sdk.files.list()):
    file.delete()

19it [00:08,  2.19it/s]
16it [00:06,  2.61it/s]
8it [00:04,  1.90it/s]
107it [00:44,  2.42it/s]
